<a href="https://colab.research.google.com/github/MiguelAngel9604/spark_exercice_1/blob/main/Pyspark_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cleaning Dates and Colum's names**

In [ ]:
# Pyspark
!pip install pyspark
# Pyspark stubs
!pip install pyspark-stubs

In [3]:
#Creating Spark session
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise2") \
    .getOrCreate()

In [58]:
df = spark.read.format("csv").option("header","true").load("/content/Data exercise 2 - Hoja 1 (2).csv")
df.show()
df.printSchema()

+------------+-----------+
|it's my name| birth/dob.|
+------------+-----------+
|       angel| 03-09-2020|
|      miguel| 03-08-2021|
|        jose|02-Aug-2022|
|        raul|29March2022|
|       pedro|Mar/2020/22|
+------------+-----------+

root
 |-- it's my name: string (nullable = true)
 |-- birth/dob.: string (nullable = true)



In [59]:
#Clean data colums names
import re
cols = [re.sub('[^a-zA-Z0-9]','',c) for c in df.columns]
print(cols)
#Create new data frame within the new column's names
new_df = df.toDF(*cols)
new_df.show()


['itsmyname', 'birthdob']
+---------+-----------+
|itsmyname|   birthdob|
+---------+-----------+
|    angel| 03-09-2020|
|   miguel| 03-08-2021|
|     jose|02-Aug-2022|
|     raul|29March2022|
|    pedro|Mar/2020/22|
+---------+-----------+



In [60]:
#Create function that will return a new date 
#This function will return the first columns that is not null using the coalesce, will iterathe over the array of strings and verify the date format
def dynamic_date(col,frmts=("yyyy-MM-dd","dd-MMM-yyyy","ddMMMMyyyy","MM-dd-yyyy","MMM/yyyy/dd","dd-M-yy")):

  return coalesce(*[to_date(col,i) for i in frmts])

res = new_df.withColumn("birth_new",dynamic_date(col("birthdob")))  
res.show()
#res.show()

+---------+-----------+----------+
|itsmyname|   birthdob| birth_new|
+---------+-----------+----------+
|    angel| 03-09-2020|2020-03-09|
|   miguel| 03-08-2021|2021-03-08|
|     jose|02-Aug-2022|2022-08-02|
|     raul|29March2022|2022-03-29|
|    pedro|Mar/2020/22|2020-03-22|
+---------+-----------+----------+



In [76]:
#data process
res.createOrReplaceTempView("tab")

#result = spark.sql("select * from tab where birth_new > '2022-03-07' ")
result = spark.sql("select * from tab where birth_new >'2021-03-07'")
result.show()


+---------+-----------+----------+
|itsmyname|   birthdob| birth_new|
+---------+-----------+----------+
|   miguel| 03-08-2021|2021-03-08|
|     jose|02-Aug-2022|2022-08-02|
|     raul|29March2022|2022-03-29|
+---------+-----------+----------+

